Implement a feed forward neural network with three hidden layers for the CIFAR-10 dataset.Train the network using a baseline optimization algorithm, such as stochastic gradient descent (SGD) or Adam, without any specific weight initialization technique or regularization technique. Record the accuracy and loss during training.

(a) Repeat the training process with Xavier initialization for weight initialization. Compare the convergence speed and accuracy of the network with the baseline results. Analyze the impact of Xavier initialization on the network's performance.

(b) Repeat the training process with Kaiming initialization for weight initialization. Compare the convergence speed and accuracy of the network with the baseline results. Analyze the impact of Kaiming initialization on the network's performance.

(c) Implement dropout regularization by applying dropout to the hidden layers of the network. Train the network with dropout regularization and compare its performance with the baseline results. Analyze the impact of dropout on the network's performance in terms of accuracy and overfitting

(d) Implement L1 or L2 regularization techniques by adding a regularization term to the loss function during training. Train the network with regularization and compare its performance with the baseline results. Analyze the impact of regularization on the network's performance in terms of accuracy and prevention of overfitting.

# BASE MODEL

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import numpy as np
import time

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = to_categorical(y_train), to_categorical(y_test) # Convert labels to one-hot encoding

In [3]:
base_model = models.Sequential([
    layers.Flatten(input_shape=(32, 32, 3)),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

base_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

st = time.time()
base_model_history = base_model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))
sp = time.time()

print("- -" * 40)

base_test_loss, base_test_acc = base_model.evaluate(x_test, y_test)
base_time_taken = round(sp-st, 2)
print(f"Base Model Results:\n\nTest Loss: {round(base_test_loss, 4)}\nTest Accuracy: {round(base_test_acc * 100, 2)}%\nTime Taken: {base_time_taken} seconds")

Epoch 1/5
782/782 [==============================] - 11s 12ms/step - loss: 1.8735 - accuracy: 0.3209 - val_loss: 1.7488 - val_accuracy: 0.3789
Epoch 2/5
782/782 [==============================] - 8s 11ms/step - loss: 1.6902 - accuracy: 0.3891 - val_loss: 1.6526 - val_accuracy: 0.4119
Epoch 3/5
782/782 [==============================] - 8s 11ms/step - loss: 1.6038 - accuracy: 0.4251 - val_loss: 1.6066 - val_accuracy: 0.4229
Epoch 4/5
782/782 [==============================] - 8s 11ms/step - loss: 1.5516 - accuracy: 0.4471 - val_loss: 1.5462 - val_accuracy: 0.4506
Epoch 5/5
782/782 [==============================] - 8s 11ms/step - loss: 1.5030 - accuracy: 0.4634 - val_loss: 1.5168 - val_accuracy: 0.4530
- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -
313/313 [==============================] - 1s 3ms/step - loss: 1.5168 - accuracy: 0.4530
Base Model Results:

Test Loss: 1.5168
Test Accuracy: 45.3%
Time Taken: 44.28 s

# MODELS WITH KERNEL INITIALIZERS
Xavier & Kaiming Weight Initializers

In [4]:
def create_model(initializer=None):
    model_with_kernel = models.Sequential([
        layers.Flatten(input_shape=(32, 32, 3)),
        layers.Dense(256, activation='relu', kernel_initializer=initializer),
        layers.Dense(128, activation='relu', kernel_initializer=initializer),
        layers.Dense(64, activation='relu', kernel_initializer=initializer),
        layers.Dense(10, activation='softmax')
    ])
    return model_with_kernel

In [5]:
res = []

# glorot_uniform - Xavier Weight Initializer, he_normal - Kaiming Weight Initializer
weight_initializers = ['glorot_uniform', 'he_normal']

for init in weight_initializers:
    model = create_model(initializer=init)
    model.compile(optimizer='sgd',  loss='categorical_crossentropy', metrics=['accuracy'])

    wei_st = time.time()
    model.fit(x_train, y_train, epochs=5)
    wei_sp = time.time()

    test_loss, test_accuracy = model.evaluate(x_test, y_test)

    time_taken = wei_sp - wei_st
    res.append((time_taken, test_accuracy))

    print("- -" * 40)

    print(f"Training with initializer: {init}\n")
    print(f"\nTest Accuracy (using '{init}' weight initialization): {round(test_accuracy * 100, 2)}%\n\nTime Taken: {round(time_taken, 2)} seconds\n")

    print("- -" * 40)

# Comparison with baseline results
base_glorot_acc = round(base_test_acc / res[0][1], 2)
base_glorot_time = round(base_time_taken / res[0][0], 3)
base_he_acc = round(base_test_acc / res[1][1])
base_he_time = round(base_time_taken / res[1][0], 3)

print(f"Baseline Results Comparison:\n\nAccuracy comparison:\n{weight_initializers[0]}: {base_glorot_acc}% accurate than base model\n{weight_initializers[1]}: {base_he_acc}% accurate than base model")
print(f"\nTime comparison:\n{weight_initializers[0]}: {base_glorot_time} seconds faster than base model\n{weight_initializers[1]}: {base_he_time} seconds faster than base model")

Epoch 1/5
1563/1563 [==============================] - 8s 5ms/step - loss: 1.8967 - accuracy: 0.3164
Epoch 2/5
1563/1563 [==============================] - 7s 4ms/step - loss: 1.7073 - accuracy: 0.3883
Epoch 3/5
1563/1563 [==============================] - 7s 5ms/step - loss: 1.6206 - accuracy: 0.4240
Epoch 4/5
1563/1563 [==============================] - 7s 5ms/step - loss: 1.5630 - accuracy: 0.4432
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 1.7241 - accuracy: 0.4025
Training with initializer: glorot_uniform


Test Accuracy (using 'glorot_uniform' weight initialization): 40.25%

Time Taken: 38.01 seconds

- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -
Epoch 1/5
1563/1563 [==============================] - 8s 5ms/step - loss: 1.8771 - accuracy: 0.3225
Epoch 2/5
1563/1563 [==============================] - 7s 5ms/step - loss: 1.6890 - accuracy: 0.3979
Epoch 3/5
1563/1563 [============

# MODEL WITH DROPOUT LAYER
Dropout Rate - 0.2

In [6]:
model_with_dropout = models.Sequential([
      layers.Flatten(input_shape=(32, 32, 3)),
      layers.Dense(256, activation='relu'),
      layers.Dropout(0.2),
      layers.Dense(128, activation='relu'),
      layers.Dropout(0.2),
      layers.Dense(64, activation='relu'),
      layers.Dropout(0.2),
      layers.Dense(10, activation='softmax')
  ])

# Load and preprocess CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


model_with_dropout.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_with_dropout.fit(x_train, y_train, epochs=5)

dropout_test_loss, dropout_test_accuracy = model_with_dropout.evaluate(x_test, y_test)

print("- -" * 40)

# Comparison with baseline results
base_dropout_acc = round(base_test_acc / dropout_test_accuracy , 2)

print("Dropout Layer Results:\n")
print(f"\nTest Accuracy: {round(dropout_test_accuracy * 100, 4)}%")
print(f"Baseline Results Comparison:\nAccuracy: {base_dropout_acc}% faster than base model")

Epoch 1/5
1563/1563 [==============================] - 8s 5ms/step - loss: 2.0559 - accuracy: 0.2447
Epoch 2/5
1563/1563 [==============================] - 8s 5ms/step - loss: 1.8754 - accuracy: 0.3246
Epoch 3/5
1563/1563 [==============================] - 7s 5ms/step - loss: 1.8042 - accuracy: 0.3503
Epoch 4/5
1563/1563 [==============================] - 7s 5ms/step - loss: 1.7585 - accuracy: 0.3709
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 1.6366 - accuracy: 0.4134
- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -
Dropout Layer Results:


Test Accuracy: 41.34%
Baseline Results Comparison:
Accuracy: 1.1% faster than base model


# MODEL WITH KERNEL REGULARIZER
L2 Kernel Regularizer

In [7]:
model_with_reg = models.Sequential([
    layers.Flatten(input_shape=(32, 32, 3)),
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dense(10, activation='softmax')
])

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model_with_reg.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_with_reg.fit(x_train, y_train, epochs=5)

reg_test_loss, reg_test_accuracy = model_with_reg.evaluate(x_test, y_test)

print("- -" * 40)

# Comparison with baseline results
base_reg_acc = round(base_test_acc / reg_test_accuracy , 2)

print("L2 Regularizer Results:\n")
print(f"\nTest Accuracy : {round(reg_test_accuracy * 100, 4)}%")
print(f"Baseline Results Comparison:\nAccuracy: {base_reg_acc}% faster than base model")

Epoch 1/5
1563/1563 [==============================] - 8s 5ms/step - loss: 7.3598 - accuracy: 0.3066
Epoch 2/5
1563/1563 [==============================] - 8s 5ms/step - loss: 4.7534 - accuracy: 0.3682
Epoch 3/5
1563/1563 [==============================] - 8s 5ms/step - loss: 3.3996 - accuracy: 0.3890
Epoch 4/5
1563/1563 [==============================] - 8s 5ms/step - loss: 2.6734 - accuracy: 0.3963
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 2.2554 - accuracy: 0.3762
- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -
L2 Regularizer Results:


Test Accuracy : 37.62%
Baseline Results Comparison:
Accuracy: 1.2% faster than base model
